In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import HTML

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)

### Article Scraping

In [3]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
# set up the HTML parser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
# get the title
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8641/nasas-perseverance-mars-rover-gets-its-wheels-and-air-brakes/" target="_self">NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes</a></div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.'

### Image Scraping

In [8]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [10]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [12]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA18846_hires.jpg'

In [13]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18846_hires.jpg'

### Table Scraping

In [14]:
df = pd.read_html('http://space-facts.com/mars/')[0]
len(df.columns)

2

In [15]:
# use Pandas to get html table
df = pd.read_html('http://space-facts.com/mars/')[0]

if len(df.columns)==2:
    df.columns=['Description', 'Mars']
else:
    df.columns=['Description', 'Mars', 'Earth']
        

df.set_index('Description', inplace=True)
df.index.name = None
df

,Mars
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
df.to_html(justify='center', classes='table table-striped table table-hover table-condensed table-responsive')

'<table border="1" class="dataframe table table-striped table table-hover table-condensed table-responsive">\n  <thead>\n    <tr style="text-align: center;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    <

### Challenge Scrapes

In [17]:
# Visit URL
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
browser.is_element_present_by_xpath("//div[@class='description']/a/h3", wait_time=1)

True

In [18]:
# Go to the first image
image_1 = browser.find_by_xpath("//div[@class='description']/a/h3")[0].click()
browser.is_element_present_by_css("img.wide-image", wait_time=1)

True

In [19]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [20]:
# Find the relative image url
img_url_rel_1 = img_soup.select_one('img.wide-image').get("src")
img_url_rel_1

'/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'

In [21]:
# Use the base URL to create an absolute URL
img_url_1 = f'https://astrogeology.usgs.gov{img_url_rel_1}'
img_url_1

'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'

In [22]:
# Get the title for the image
title_1 = img_soup.select_one('h2.title').get_text()
title_1

'Cerberus Hemisphere Enhanced'

In [23]:
# Find the relative image url
img_thumb_rel_1 = img_soup.select_one('img.thumb').get("src")
img_thumb_rel_1

'/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png'

In [24]:
# Use the base URL to create an absolute URL
img_thumb_1 = f'https://astrogeology.usgs.gov{img_thumb_rel_1}'
img_thumb_1

'https://astrogeology.usgs.gov/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png'

In [25]:
# Get the description for the image
description = img_soup.select_one('p').get_text()
description

'Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired on February 11, 1980. At that time, it was early northern summer on Mars. The center of the image is at latitude 3 degrees, longitude 185 degrees.'

In [26]:
# Go back to site
browser.back()
# Optional delay for loading the page
browser.is_element_present_by_xpath("//div[@class='description']/a/h3", wait_time=1)

True

In [186]:
# Go to the second image
image_2 = browser.find_by_xpath("//div[@class='description']/a/h3")[1].click()
browser.is_element_present_by_css("img.wide-image", wait_time=1)

True

In [187]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [188]:
# Find the relative image url
img_url_rel_2 = img_soup.select_one('img.wide-image').get("src")
img_url_rel_2

'/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'

In [189]:
# Use the base URL to create an absolute URL
img_url_2 = f'https://astrogeology.usgs.gov{img_url_rel_2}'
img_url_2

'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'

In [190]:
# Get the title for the image
title_2 = img_soup.select_one('h2.title').get_text()
title_2

'Schiaparelli Hemisphere Enhanced'

In [191]:
# Go back to site
browser.back()
# Optional delay for loading the page
browser.is_element_present_by_xpath("//div[@class='description']/a/h3", wait_time=1)

True

In [192]:
# Go to the third image
image_3 = browser.find_by_xpath("//div[@class='description']/a/h3")[2].click()
browser.is_element_present_by_css("img.wide-image", wait_time=1)

True

In [193]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [142]:
# Find the relative image url
img_url_rel_3 = img_soup.select_one('img.wide-image').get("src")
img_url_rel_3

'/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'

In [143]:
# Use the base URL to create an absolute URL
img_url_3 = f'https://astrogeology.usgs.gov{img_url_rel_3}'
img_url_3

'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'

In [144]:
# Get the title for the image
title_3 = img_soup.select_one('h2.title').get_text()
title_3

'Syrtis Major Hemisphere Enhanced'

In [145]:
# Go back to site
browser.back()
# Optional delay for loading the page
browser.is_element_present_by_xpath("//div[@class='description']/a/h3", wait_time=1)

True

In [146]:
# Go to the fourth image
image_4 = browser.find_by_xpath("//div[@class='description']/a/h3")[3].click()
browser.is_element_present_by_css("img.wide-image", wait_time=1)

True

In [147]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [148]:
# Find the relative image url
img_url_rel_4 = img_soup.select_one('img.wide-image').get("src")
img_url_rel_4

'/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'

In [149]:
# Use the base URL to create an absolute URL
img_url_4 = f'https://astrogeology.usgs.gov{img_url_rel_4}'
img_url_4

'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'

In [150]:
# Get the title for the image
title_4 = img_soup.select_one('h2.title').get_text()
title_4

'Valles Marineris Hemisphere Enhanced'

In [151]:
# Go back to site
browser.back()
# Optional delay for loading the page
browser.is_element_present_by_xpath("//div[@class='description']/a/h3", wait_time=1)

True

In [207]:
# function to pull Mars Hemishpheres
def mars_hemishpheres(browser):
    
    # Visit URL
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    # Add wait time for page to load
    browser.is_element_present_by_xpath("//div[@class='description']/a/h3", wait_time=1)
    
    mars_hemi_list = []
    
    # function to scrape images and title
    def scrape_hemi(index_num):
            
        # click on link for hemishpere site
        full_image = browser.find_by_xpath("//div[@class='description']/a/h3")[index_num].click()
        # add wait
        browser.is_element_present_by_css("img.wide-image", wait_time=1)
            
        # Parse the resulting html with soup
        html = browser.html
        img_soup = BeautifulSoup(html, 'html.parser')
            
        # Get the title for the image
        title = img_soup.select_one('h2.title').get_text()
            
        # Find the relative image url
        img_url_rel_full = img_soup.select_one('img.wide-image').get("src")
            
        # Use the base URL to create an absolute URL
        img_url_full = f'https://astrogeology.usgs.gov{img_url_rel_full}'
            
        # Find the relative thumb image url
        img_thumb_rel = img_soup.select_one('img.thumb').get("src")
            
        # Use the base URL to create an absolute URL
        img_thumb = f'https://astrogeology.usgs.gov{img_thumb_rel}'
        
        # Go back to site
        browser.back()
        # Optional delay for loading the page
        browser.is_element_present_by_xpath("//div[@class='description']/a/h3", wait_time=1)
        
        hemi_results = {"title": title, "img_url": img_url_full, "img_thumb": img_thumb}
        
            
        return mars_hemi_list.append(hemi_results)
        
    # run scape_hemi function for four images and titles
    for index_num in range(0,4):
        scrape_hemi(index_num)
        
    return mars_hemi_list


In [208]:
mars_hemi_list = mars_hemishpheres(browser)
mars_hemi_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'img_thumb': 'https://astrogeology.usgs.gov/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'img_thumb': 'https://astrogeology.usgs.gov/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'img_thumb': 'https://astrogeology.usgs.gov/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/ca

In [27]:
browser.quit()

In [211]:
mars_hemi_list[0]['title']

'Cerberus Hemisphere Enhanced'